Tokenizing cells for Geneformer, which can be used to predict cell types in the uninfected data from the time course data

In [14]:
import sys

sys.path.append("/home/amonell/Geneformer")
from geneformer import TranscriptomeTokenizer
import numpy as np
import os
import pandas as pd

Put the path to the reference data

In [ ]:
import scanpy as sc

ad = sc.read(
    "timecourse.h5ad"
)

#### Converting gene names to human homolog ensembl ids
We provide the following file in the current directory

In [17]:
ensembl = pd.read_csv("gProfiler_hsapiens_9-18-2023_2-31-49 PM.csv").drop_duplicates(
    "initial_alias"
)

In [18]:
ad.var["ensembl_id"] = [i for i in ensembl["converted_alias"] if not pd.isna(i)][
    : len(ad.var.index)
]
ad.var.index = ad.var.ensembl_id.tolist()
ad.obs["n_counts"] = np.array(np.sum(ad.X, axis=1)).flatten()
ad.obs["organ_major"] = "SI"

In [19]:
ad.obs["input_ids"] = [i for i in range(len(ad.obs.index.tolist()))]
d = {}
for i in ad.var.index.tolist():
    d[i] = True


In [20]:
tk = TranscriptomeTokenizer(
    {"Subtype": "cell_type", "organ_major": "organ_major"}, nproc=4
)


### Tokenize datasets to annotate

In [ ]:
import glob

geneformer_dir = "/mnt/sata1/Analysis_Alex/Geneformer"

ensembl = pd.read_csv("gProfiler_hsapiens_9-18-2023_2-31-49 PM.csv").drop_duplicates(
    "initial_alias"
)

for filename in glob.glob("/mnt/sata1/Analysis_Alex/uninfected/segmentation_SI*"):
    outname = os.path.basename(filename)
    fname = os.path.dirname(filename)
    try:
        os.mkdir(os.path.join(geneformer_dir, "loom_" + outname))
        ad = sc.read(
            os.path.join(
                fname,
                outname,
                "adatas/06_reference_mapped.h5ad",
            )
        )
        ad.var["ensembl_id"] = [
            i for i in ensembl["converted_alias"] if not pd.isna(i)
        ][: len(ad.var.index)]
        ad.var.index = ad.var.ensembl_id.tolist()
        ad.obs["n_counts"] = np.array(np.sum(ad.X, axis=1)).flatten()
        ad.obs["organ_major"] = "SI"
        ad.obs["Subtype"] = 0
        ad.write_loom(
            os.path.join(geneformer_dir, "loom_"
            + outname
            + "/"
            + outname
            + ".loom")
        )

        tk = TranscriptomeTokenizer(
            {"Subtype": "cell_type", "organ_major": "organ_major"}, nproc=4
        )
        tk.tokenize_data(
            os.path.join(geneformer_dir,"loom_" + outname),
            os.path.join(geneformer_dir,"loom_" + outname, "tokenized"),
            "train_" + outname,
        )
    except:
        print(outname)